In [1]:
import pandas as pd
import plotly.graph_objects as go
from utils import *

co_df = pd.read_csv("data/co_data.csv")
map_df = pd.read_csv("data/map_data.csv")
user_df = pd.read_csv("data/user_data.csv", index_col= 'userName')
replay_df = pd.read_csv("data/replay_data.csv", index_col= 'replayId')
pick_df = pd.read_csv("data/pick_data.csv", index_col= 'replayId')
ban_df = pd.read_csv("data/ban_data.csv", index_col= 'replayId')

In [2]:
# Filter high ranked players, >= 1400 represents 25%, top quartile
highranked_users = user_df[(user_df['stdScore'] >= 1400) | (user_df['fogScore'] >= 1400)]

# Replays played by high ranked players, exclude time out and draws
highranked_replays = replay_df.loc[pick_df[pick_df['userName'].isin(highranked_users.index)].index.unique()]
highranked_replays = highranked_replays[(highranked_replays['isBoot'] != True) & 
                                        (highranked_replays['winnerPlayerIndex'] != 0)]

In [3]:
CO_NAME = 'Gage'

available_matches = get_co_available_replays(CO_NAME, ban_df, highranked_replays.index)
banned_matches = get_co_banned_replays(CO_NAME, ban_df, highranked_replays.index)
picked_matches = get_co_picked_replays(CO_NAME, pick_df, highranked_replays.index)

pickrate = get_co_pick_rate(CO_NAME, pick_df, ban_df, highranked_replays.index)
banrate = get_co_ban_rate(CO_NAME, ban_df, highranked_replays.index)


In [4]:
# need to filter total
co_matchup_total = get_co_matchup_total(CO_NAME, pick_df, highranked_replays.index)

# High ranked matches where CO of interest is picked but is not the winner
# Excludes mirror matchups, counts losses
# Should make function for matchup_loss
co_matchup_winner = highranked_replays.loc[picked_matches, :]
co_matchup_loss = co_matchup_winner[co_matchup_winner['winnerCoName'] != CO_NAME]['winnerCoName'].value_counts()

In [5]:
# Need better way to count mirror matchups
co_matchup_wins = co_matchup_total - co_matchup_loss
co_matchup_wins = co_matchup_wins.fillna(1)
co_matchup_wins = co_matchup_wins.rename("Wins")

co_matchup_winrate = co_matchup_wins / co_matchup_total
co_matchup_winrate = co_matchup_winrate.rename("Win Rate")

co_matchup_table = pd.concat([co_matchup_wins, co_matchup_total, co_matchup_winrate], axis=1)
co_matchup_table = co_matchup_table.sort_values(['matchupTotal'], ascending= False)
co_matchup_table.index = co_matchup_table.index.rename(CO_NAME + " vs")
co_matchup_table

,Wins,matchupTotal,Win Rate
Gage vs,,,
Will,3.0,9,0.333333
Lin,1.0,9,0.111111
Tabitha,2.0,9,0.222222
Hawk,1.0,8,0.125000
Caulder,2.0,7,0.285714
Penny,0.0,6,0.000000
Grat,2.0,5,0.400000
Greyfield,3.0,5,0.600000
Forsythe,1.0,4,0.250000


In [6]:
# Why can't gage win vs penny?
# This is an abomination
pick_df.loc[pick_df.loc[picked_matches, :][pick_df.loc[picked_matches, :]['coName'] == 'Penny'].index, :]

,playerIndex,coName,userName
replayId,,,
6017,1,Penny,Plague
6017,2,Gage,Mini Nini
5719,1,Gage,PhantomFullForce
5719,2,Penny,Sliicer
5078,1,Gage,BWAVE
5078,2,Penny,BusterBeachside
4789,1,Gage,Chan ' tcho
4789,2,Penny,DC290
4558,1,Gage,Tee Lord


In [7]:
replay_df.loc[4169]

winnerPlayerIndex             1
turns                        18
actions                     835
mapId                       110
isStd                      True
isBoot                    False
mapName              Swamp City
winnerCoName               Gage
winnerUserName           Mauwus
Name: 4169, dtype: object